In [ ]:
import numpy as np
import sklearn
import matplotlib.pyplot as plt

### 1. Jednostavna regresija

Zadan je skup primjera $\mathcal{D}=\{(x^{(i)},y^{(i)})\}_{i=1}^4 = \{(0,4),(1,1),(2,2),(4,5)\}$. Primjere predstavite matricom $\mathbf{X}$ dimenzija $N\times n$ (u ovom slučaju $4\times 1$) i vektorom oznaka $\textbf{y}$, dimenzija $N\times 1$ (u ovom slučaju $4\times 1$), na sljedeći način:


In [ ]:
X = np.array([[0],[1],[2],[4]])
y = np.array([4,1,2,5])

### (a)

Proučite funkciju [`PolynomialFeatures`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) iz biblioteke `sklearn` i upotrijebite je za generiranje matrice dizajna $\mathbf{\Phi}$ koja ne koristi preslikavanje u prostor više dimenzije (samo će svakom primjeru biti dodane *dummy* jedinice; $m=n+1$).


In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(1)
phi = poly.fit_transform(X)
print(phi)

### (b)

Upoznajte se s modulom [`linalg`](http://docs.scipy.org/doc/numpy/reference/routines.linalg.html). Izračunajte težine $\mathbf{w}$ modela linearne regresije kao $\mathbf{w}=(\mathbf{\Phi}^\intercal\mathbf{\Phi})^{-1}\mathbf{\Phi}^\intercal\mathbf{y}$. Zatim se uvjerite da isti rezultat možete dobiti izračunom pseudoinverza $\mathbf{\Phi}^+$ matrice dizajna, tj. $\mathbf{w}=\mathbf{\Phi}^+\mathbf{y}$, korištenjem funkcije [`pinv`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.pinv.html).

In [ ]:
from numpy import linalg

w = np.dot(np.dot(linalg.inv(np.dot(phi.T, phi)), phi.T), y)
print(w)
w = linalg.pinv(phi) @ y
print(w)

Radi jasnoće, u nastavku je vektor $\mathbf{x}$ s dodanom *dummy* jedinicom $x_0=1$ označen kao $\tilde{\mathbf{x}}$.

### (c)

Prikažite primjere iz $\mathcal{D}$ i funkciju $h(\tilde{\mathbf{x}})=\mathbf{w}^\intercal\tilde{\mathbf{x}}$. Izračunajte pogrešku učenja prema izrazu $E(h|\mathcal{D})=\frac{1}{2}\sum_{i=1}^N(\tilde{\mathbf{y}}^{(i)} - h(\tilde{\mathbf{x}}^{(i)}))^2$. Možete koristiti funkciju srednje kvadratne pogreške [`mean_squared_error`]( http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) iz modula [`sklearn.metrics`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics).

**Q:** Gore definirana funkcija pogreške $E(h|\mathcal{D})$ i funkcija srednje kvadratne pogreške nisu posve identične. U čemu je razlika? Koja je "realnija"?

In [ ]:
from sklearn.metrics import mean_squared_error

#h(x) = w^T x
for x in phi:
    print(f'h({x}) = {w @ x}')

h = phi @ w
print(f'h = {h}')

error = 0
for i in range(4):
    error += (y[i] - h[i])**2
error *= 0.5

print(f'E(h|D) = {error}')

print(f'Mean squared error = {mean_squared_error(y, h)}')

### (d)

Uvjerite se da za primjere iz $\mathcal{D}$ težine $\mathbf{w}$ ne možemo naći rješavanjem sustava $\mathbf{w}=\mathbf{\Phi}^{-1}\mathbf{y}$, već da nam doista treba pseudoinverz.

**Q:** Zašto je to slučaj? Bi li se problem mogao riješiti preslikavanjem primjera u višu dimenziju? Ako da, bi li to uvijek funkcioniralo, neovisno o skupu primjera $\mathcal{D}$? Pokažite na primjeru.

In [ ]:
try:
    w = linalg.inv(phi) @ y
except:
    print("Ne ide")

### (e) 

Proučite klasu [`LinearRegression`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) iz modula [`sklearn.linear_model`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model). Provjerite jesu li težine koje izračunava ta funkcija (dostupne pomoću atributa `coef_` i `intercept_`) jednake onima koje ste izračunali gore. Ako nisu, prilagodite kôd tako da jest.

**NB:** Obratite pozornost na to kako klase [`LinearRegression`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) i [`PolynomialFeatures`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) koriste pomak i osigurajte da ga ne dodajete više puta.

Izračunajte predikcije modela (metoda `predict`) i uvjerite se da je pogreška učenja identična onoj koju ste ranije izračunali.

In [ ]:
from sklearn.linear_model import LinearRegression

rez = LinearRegression().fit(phi, y)
print(rez.score(phi, y))
print(rez.coef_)
print(rez.intercept_)
print(rez.predict(np.array([[1, 2]])))

### 2. Polinomijalna regresija i utjecaj šuma

### (a)

Razmotrimo sada regresiju na većem broju primjera. Definirajte funkciju `make_labels(X, f, noise=0)` koja uzima matricu neoznačenih primjera $\mathbf{X}_{N\times n}$ te generira vektor njihovih oznaka $\mathbf{y}_{N\times 1}$. Oznake se generiraju kao $y^{(i)} = f(x^{(i)})+\mathcal{N}(0,\sigma^2)$, gdje je $f:\mathbb{R}^n\to\mathbb{R}$ stvarna funkcija koja je generirala podatke (koja nam je u stvarnosti nepoznata), a $\sigma$ je standardna devijacija Gaussovog šuma, definirana parametrom `noise`. Za generiranje šuma možete koristiti funkciju [`numpy.random.normal`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html). 

Generirajte skup za učenje od $N=50$ primjera uniformno distribuiranih u intervalu $[-5,5]$ pomoću funkcije $f(x) = 5 + x -2 x^2 -5 x^3$ uz šum  $\sigma=200$:

In [ ]:
from numpy.random import normal
def make_labels(X, f, noise=0):
    rez = []
    for i in X:
        rez.append(f(i) + normal(0, noise))
    return np.array(rez)

In [ ]:
def make_instances(x1, x2, N) :
    return np.array([np.array([x]) for x in np.linspace(x1,x2,N)])

In [ ]:
a = make_instances(-5, 5, 50)
b = make_labels(a, lambda x: 5 + x - 2 * x**2 - 5 * x**3, 200)

Prikažite taj skup funkcijom [`scatter`](http://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.scatter).

In [ ]:
plt.scatter(a, b)
plt.show()

### (b)

Trenirajte model polinomijalne regresije stupnja $d=3$. Na istom grafikonu prikažite naučeni model $h(\mathbf{x})=\mathbf{w}^\intercal\tilde{\mathbf{x}}$ i primjere za učenje. Izračunajte pogrešku učenja modela.

In [ ]:
model = LinearRegression()
novi = PolynomialFeatures(3).fit_transform(a.reshape(-1, 1))
model.fit(novi, b)
predict = model.predict(novi)
print(f'Error: {mean_squared_error(b, predict)}')
print(model.coef_)
plt.plot(a, predict)
plt.scatter(a, b)
plt.show()

### 3. Odabir modela

### (a)

Na skupu podataka iz zadatka 2 trenirajte pet modela linearne regresije $\mathcal{H}_d$ različite složenosti, gdje je $d$ stupanj polinoma, $d\in\{1,3,5,10,20\}$. Prikažite na istome grafikonu skup za učenje i funkcije $h_d(\mathbf{x})$ za svih pet modela (preporučujemo koristiti `plot` unutar `for` petlje). Izračunajte pogrešku učenja svakog od modela.

**Q:** Koji model ima najmanju pogrešku učenja i zašto?

In [ ]:
stupanj = [1, 3, 5, 10, 20]
for d in stupanj:
    novi = PolynomialFeatures(d).fit_transform(a.reshape(-1, 1))
    model.fit(novi, b)
    predict = model.predict(novi)
    print(f'Error stupnja {d}: {mean_squared_error(b, predict)}')
    plt.plot(a, predict)
plt.scatter(a, b)
plt.show()

### (b)

Razdvojite skup primjera iz zadatka 2 pomoću funkcije [`model_selection.train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) na skup za učenja i skup za ispitivanje u omjeru 1:1. Prikažite na jednom grafikonu pogrešku učenja i ispitnu pogrešku za modele polinomijalne regresije $\mathcal{H}_d$, sa stupnjem polinoma $d$ u rasponu $d\in [1,2,\ldots,20]$. Budući da kvadratna pogreška brzo raste za veće stupnjeve polinoma, umjesto da iscrtate izravno iznose pogrešaka, iscrtajte njihove logaritme.

**NB:** Podjela na skupa za učenje i skup za ispitivanje mora za svih pet modela biti identična.

**Q:** Je li rezultat u skladu s očekivanjima? Koji biste model odabrali i zašto?

**Q:** Pokrenite iscrtavanje više puta. U čemu je problem? Bi li problem bio jednako izražen kad bismo imali više primjera? Zašto?

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(a, b, test_size=0.5, random_state=83)
stupanj = [x for x in range(1, 21)]
greske_train = []
greske_test = []

for d in stupanj:

    novi_train = PolynomialFeatures(d).fit_transform(X_train)
    model.fit(novi_train, y_train)
    predict = model.predict(novi_train)
    greske_train.append(mean_squared_error(y_train, predict))
    #if d % 5 == 0:
        #print(f'Greska train: {np.log(mean_squared_error(y_train, predict))}')

    novi_test = PolynomialFeatures(d).fit_transform(X_test)
    predict = model.predict(novi_test)
    greske_test.append(mean_squared_error(y_test, predict))
    #if d % 5 == 0:
        #print(f'Greska test: {np.log(mean_squared_error(y_test, predict))}')

#greske = np.log([i[1]/i[0] for i in zip(greske_train, greske_test)]).reshape(-1, 1)
#print(greske)

plt.plot(np.array(stupanj).reshape(-1, 1), np.log(np.array(greske_train).reshape(-1, 1)))
plt.plot(np.array(stupanj).reshape(-1, 1), np.log(np.array(greske_test).reshape(-1, 1)))
plt.show()

### (c)

Točnost modela ovisi o (1) njegovoj složenosti (stupanj $d$ polinoma), (2) broju primjera $N$, i (3) količini šuma. Kako biste to analizirali, nacrtajte grafikone pogrešaka kao u 3b, ali za različit $N\in$ (trećina, dvije trećine, sve) i količine šuma $\sigma\in\{100,200,500\}$ (ukupno 9 grafikona). Upotrijebite funkciju [`subplots`](http://matplotlib.org/examples/pylab_examples/subplots_demo.html) kako biste pregledno posložili grafikone u tablicu $3\times 3$. Podatci se generiraju na isti način kao u zadatku 2.

**NB:** Pobrinite se da svi grafikoni budu generirani nad usporedivim skupovima podataka, na sljedeći način. Generirajte najprije svih 1000 primjera, podijelite ih na skupove za učenje i skupove za ispitivanje (dva skupa od po 500 primjera). Zatim i od skupa za učenje i od skupa za ispitivanje načinite tri različite verzije, svaka s drugačijom količinom šuma (ukupno 2x3=6 verzija podataka). Kako bi simulirali veličinu skupa podataka, od tih dobivenih 6 skupova podataka uzorkujte trećinu, dvije trećine i sve podatke. Time ste dobili 18 skupova podataka -- skup za učenje i za testiranje za svaki od devet grafova.

In [ ]:
X = make_instances(-5, 5, 1000)
X_train, X_test = train_test_split(X, test_size=0.5, random_state=1745)

y_train = []
y_test = []

for noise in [100, 200, 500]:
    y_train.append(make_labels(X_train, lambda x: 5 + x - 2 * x**2 - 5 * x**3, noise))
    y_test.append(make_labels(X_test, lambda x: 5 + x - 2 * x**2 - 5 * x**3, noise))

y_train = np.array(y_train)
y_test = np.array(y_test)

data_train = []
data_test = []

for i in [1/3, 2/3, 1]:
    
    for y in y_train:

        data_train.append((X_train[:int(i * len(X_train))], y[:int(i * len(X_train))]))

    for y in y_test:

        data_test.append((X_test[:int(i * len(X_test))], y[:int(i * len(X_test))]))

errors = []

for ((x_train, y_train), (x_test, y_test)) in zip(data_train, data_test): #((x_train, y_train), (x_test, y_test))

    errors_train = []
    errors_test = []
    
    for d in stupanj:

        novi_train = PolynomialFeatures(d).fit_transform(x_train)
        model.fit(novi_train, y_train)
        predict_train = model.predict(novi_train)
        errors_train.append(mean_squared_error(y_train, predict_train))

        novi_test = PolynomialFeatures(d).fit_transform(x_test)
        predict_test = model.predict(novi_test)
        errors_test.append(mean_squared_error(y_test, predict_test))

    errors_train = np.log(np.array(errors_train))
    errors_test = np.log(np.array(errors_test))

    errors.append((errors_train, errors_test, (len(x_train))))

f, axarr = plt.subplots(3, 3)
a = 0
b = 0

for i in errors:

    axarr[a, b].plot(stupanj, i[0])
    axarr[a, b].plot(stupanj, i[1])
    axarr[a, b].set_title(f'N: {i[2]}')

    b += 1

    if b == 3:
        a += 1
        b = 0

plt.show()

"""
f, axarr = plt.subplots(3, 3)
axarr[0, 0].plot(x, y)
axarr[0, 0].set_title('Axis [0,0]')
axarr[0, 1].scatter(x, y)
axarr[0, 1].set_title('Axis [0,1]')
axarr[1, 0].plot(x, y ** 2)
axarr[1, 0].set_title('Axis [1,0]')
axarr[1, 1].scatter(x, y ** 2)
axarr[1, 1].set_title('Axis [1,1]')
# Fine-tune figure; hide x ticks for top plots and y ticks for right plots
plt.setp([a.get_xticklabels() for a in axarr[0, :]], visible=False)
plt.setp([a.get_yticklabels() for a in axarr[:, 1]], visible=False)
"""

***Q:*** Jesu li rezultati očekivani? Obrazložite.

### 4. Regularizirana regresija

### (a)

U gornjim eksperimentima nismo koristili **regularizaciju**. Vratimo se najprije na primjer iz zadatka 1. Na primjerima iz tog zadatka izračunajte težine $\mathbf{w}$ za polinomijalni regresijski model stupnja $d=3$ uz L2-regularizaciju (tzv. *ridge regression*), prema izrazu $\mathbf{w}=(\mathbf{\Phi}^\intercal\mathbf{\Phi}+\lambda\mathbf{I})^{-1}\mathbf{\Phi}^\intercal\mathbf{y}$. Napravite izračun težina za regularizacijske faktore $\lambda=0$, $\lambda=1$ i $\lambda=10$ te usporedite dobivene težine.

**Q:** Kojih je dimenzija matrica koju treba invertirati?

**Q:** Po čemu se razlikuju dobivene težine i je li ta razlika očekivana? Obrazložite.

In [ ]:
X = np.array([[0],[1],[2],[4]])
y = np.array([4,1,2,5])

poly = PolynomialFeatures(3)
phi = poly.fit_transform(X)

w = np.linalg.inv(phi.T @ phi + 0 * np.eye(4, 4)) @ phi.T @ y
print(w)

w = np.linalg.inv(phi.T @ phi + 1 * np.eye(4, 4)) @ phi.T @ y
print(w)

w = np.linalg.inv(phi.T @ phi + 10 * np.eye(4, 4)) @ phi.T @ y
print(w)

### (b)

Proučite klasu [`Ridge`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge) iz modula [`sklearn.linear_model`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model), koja implementira L2-regularizirani regresijski model. Parametar $\alpha$ odgovara parametru $\lambda$. Primijenite model na istim primjerima kao u prethodnom zadatku i ispišite težine $\mathbf{w}$ (atributi `coef_` i `intercept_`). Ponovno, pripazite na pomak.

**Q:** Jesu li težine identične onima iz zadatka 4a? Ako nisu, objasnite zašto je to tako i kako biste to popravili.

In [ ]:
from sklearn.linear_model import Ridge
#print(phi)

model = Ridge(0)
model.fit(phi, y)
print(f'Coef i intercept za lambda 0: {model.coef_}, {model.intercept_}')

model = Ridge(1)
model.fit(phi, y)
print(f'Coef i intercept za lambda 1: {model.coef_}, {model.intercept_}')

model = Ridge(10)
model.fit(phi, y)
print(f'Coef i intercept za lambda 10: {model.coef_}, {model.intercept_}')

### (c)

Vratimo se na slučaj $N=50$ slučajno generiranih primjera iz zadatka 2. Trenirajte modele polinomijalne regresije $\mathcal{H}_{\lambda,d}$ za $\lambda\in\{0,100\}$ i $d\in\{2,10\}$ (ukupno četiri modela). Skicirajte pripadne funkcije $h(\mathbf{x})$ i primjere (na jednom grafikonu; preporučujemo koristiti `plot` unutar `for` petlje).

**Q:** Jesu li rezultati očekivani? Obrazložite.

In [ ]:
lambdas = [0, 100]
degrees = [2, 10]

X = make_instances(-5, 5, 50)
y = make_labels(X, lambda x: 5 + x - 2 * x**2 - 5 * x**3, 200)

for i in lambdas:

    for d in degrees:

        polynomial_matrix = PolynomialFeatures(d).fit_transform(X)
        model = Ridge(alpha=i)
        model.fit(polynomial_matrix, y)

        predict = model.predict(polynomial_matrix)
        plt.plot(X, predict)

        print(f"Gubitak sa lambda {i} i stupnjem {d}: {mean_squared_error(y, predict)}")

plt.scatter(X, y)
plt.show()        

### (d)

Kao u zadataku 3b, razdvojite primjere na skup za učenje i skup za ispitivanje u omjeru 1:1. Prikažite krivulje logaritama pogreške učenja i ispitne pogreške u ovisnosti za model $\mathcal{H}_{d=10,\lambda}$, podešavajući faktor regularizacije $\lambda$ u rasponu $\lambda\in\{0,1,\dots,50\}$.

**Q:** Kojoj strani na grafikonu odgovara područje prenaučenosti, a kojoj podnaučenosti? Zašto?

**Q:** Koju biste vrijednosti za $\lambda$ izabrali na temelju ovih grafikona i zašto?


In [ ]:
X = make_instances(-5, 5, 50)
y = make_labels(X, lambda x: 5 + x - 2 * x**2 - 5 * x**3, 200)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=83)
stupanj = 10
lambdas = [x for x in range(1001)]
poly = PolynomialFeatures(stupanj)

error_train = []
error_test = []

for l in lambdas:

    poly_matrix_train = poly.fit_transform(X_train)
    model = Ridge(l)
    model.fit(poly_matrix_train, y_train)

    predict_train = model.predict(poly_matrix_train)
    error_train.append(mean_squared_error(y_train, predict_train))

    poly_matrix_test = poly.fit_transform(X_test)
    predict_test = model.predict(poly_matrix_test)
    error_test.append(mean_squared_error(y_test, predict_test))

plt.plot(np.array(lambdas).reshape(-1, 1), np.log(np.array(error_train).reshape(-1, 1)))
plt.plot(np.array(lambdas).reshape(-1, 1), np.log(np.array(error_test).reshape(-1, 1)))
plt.show()

## Dodatni zadatci

Zadatci u nastavku (označeni zvjezdicom) nisu dio obaveznog dijela laboratorijske vježbe, niti nose bonus bodove. Dakle, nije ih potrebno riješiti kako biste ostvarili 100% bodova na ovoj laboratorijskoj vježbi. Međutim, preporučamo vam da ih pokušate riješiti i na terminu predaje prodiskutirate svoja rješenja s asistentima.

### *5. L1-regularizacija i L2-regularizacija

Svrha regularizacije jest potiskivanje težina modela $\mathbf{w}$ prema nuli, kako bi model bio što jednostavniji. Složenost modela može se okarakterizirati normom pripadnog vektora težina $\mathbf{w}$, i to tipično L2-normom ili L1-normom. Za jednom trenirani model možemo izračunati i broj ne-nul značajki, ili L0-normu, pomoću sljedeće funkcije koja prima vektor težina $\mathbf{w}$:

In [ ]:
def nonzeroes(coef, tol=1e-6): 
    return len(coef) - len(coef[np.isclose(0, coef, atol=tol)])

### (a)

Za ovaj zadatak upotrijebite skup za učenje i skup za testiranje iz zadatka 3b. Trenirajte modele **L2-regularizirane** polinomijalne regresije stupnja $d=5$, mijenjajući hiperparametar $\lambda$ u rasponu $\{1,2,\dots,100\}$. Za svaki od treniranih modela izračunajte L{0,1,2}-norme vektora težina $\mathbf{w}$ te ih prikažite kao funkciju od $\lambda$. Pripazite što točno šaljete u funkciju za izračun normi.

**Q:** Objasnite oblik obiju krivulja. Hoće li krivulja za $\|\mathbf{w}\|_2$ doseći nulu? Zašto? Je li to problem? Zašto?

**Q:** Za $\lambda=100$, koliki je postotak težina modela jednak nuli, odnosno koliko je model rijedak?

In [ ]:
from sklearn.linear_model import Ridge

def l1_norm(x):
    x = np.abs(x)
    return np.sum(x)

def l2_norm(x):
    x = x ** 2
    x = np.sum(x)
    return np.sqrt(x)

X = make_instances(-5, 5, 50)
y = make_labels(X, lambda x: 5 + x - 2 * x**2 - 5 * x**3, 200)
poly = PolynomialFeatures(5)
X_poly = poly.fit_transform(X)
lambdas = [x for x in range(1, 101)]

l_0, l_1, l_2 = [], [], []

brojac = 0

for l in lambdas:

    model = Ridge(l)
    model.fit(X_poly, y)

    if brojac % 10 == 0:
        print(model.coef_)

    l_0.append(nonzeroes(model.coef_))
    l_1.append(l1_norm(model.coef_))
    l_2.append(l2_norm(model.coef_))

    brojac += 1

plt.plot(np.array(lambdas), np.array(l_0))
print(f'L1: {l_1}')
print(f'L2: {l_2}')
plt.plot(np.array(lambdas), np.array(l_1))
plt.plot(np.array(lambdas), np.array(l_2))
plt.show()

### (b)

Glavna prednost L1-regularizirane regresije (ili *LASSO regression*) nad L2-regulariziranom regresijom jest u tome što L1-regularizirana regresija rezultira **rijetkim modelima** (engl. *sparse models*), odnosno modelima kod kojih su mnoge težine pritegnute na nulu. Pokažite da je to doista tako, ponovivši gornji eksperiment s **L1-regulariziranom** regresijom, implementiranom u klasi  [`Lasso`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) u modulu [`sklearn.linear_model`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model).

In [ ]:
from sklearn.linear_model import Lasso

X = make_instances(-5, 5, 50)
y = make_labels(X, lambda x: 5 + x - 2 * x**2 - 5 * x**3, 200)
poly = PolynomialFeatures(5)
X_poly = poly.fit_transform(X)
lambdas = [x for x in range(1, 101)]

l_0, l_1, l_2 = [], [], []

brojac = 0
for l in lambdas:

    model = Lasso(l)
    model.fit(X_poly, y)

    if brojac % 10 == 0:
        print(model.coef_)

    l_0.append(nonzeroes(model.coef_))
    l_1.append(l1_norm(model.coef_))
    l_2.append(l2_norm(model.coef_))

    brojac += 1

plt.plot(np.array(lambdas), np.array(l_0))
print(f'L1: {l_1}')
print(f'L2: {l_2}')
plt.plot(np.array(lambdas), np.array(l_1))
plt.plot(np.array(lambdas), np.array(l_2))
plt.show()

### *6. Značajke različitih skala

Često se u praksi možemo susreti sa podatcima u kojima sve značajke nisu jednakih magnituda. Primjer jednog takvog skupa je regresijski skup podataka `grades` u kojem se predviđa prosjek ocjena studenta na studiju (1--5) na temelju dvije značajke: bodova na prijamnom ispitu (1--3000) i prosjeka ocjena u srednjoj školi. Prosjek ocjena na studiju izračunat je kao težinska suma ove dvije značajke uz dodani šum.

Koristite sljedeći kôd kako biste generirali ovaj skup podataka.

In [ ]:
n_data_points = 500
np.random.seed(69)

# Generiraj podatke o bodovima na prijamnom ispitu koristeći normalnu razdiobu i ograniči ih na interval [1, 3000].
exam_score = np.random.normal(loc=1500.0, scale = 500.0, size = n_data_points) 
exam_score = np.round(exam_score)
exam_score[exam_score > 3000] = 3000
exam_score[exam_score < 0] = 0

# Generiraj podatke o ocjenama iz srednje škole koristeći normalnu razdiobu i ograniči ih na interval [1, 5].
grade_in_highschool = np.random.normal(loc=3, scale = 2.0, size = n_data_points)
grade_in_highschool[grade_in_highschool > 5] = 5
grade_in_highschool[grade_in_highschool < 1] = 1

# Matrica dizajna.
grades_X = np.array([exam_score,grade_in_highschool]).T

# Završno, generiraj izlazne vrijednosti.
rand_noise = np.random.normal(loc=0.0, scale = 0.5, size = n_data_points)
exam_influence = 0.9
grades_y = ((exam_score / 3000.0) * (exam_influence) + (grade_in_highschool / 5.0) \
            * (1.0 - exam_influence)) * 5.0 + rand_noise
grades_y[grades_y < 1] = 1
grades_y[grades_y > 5] = 5

### a)

Iscrtajte ovisnost ciljne vrijednosti (y-os) o prvoj i o drugoj značajki (x-os). Iscrtajte dva odvojena grafa.

In [ ]:
f, axarr = plt.subplots(1, 2)
axarr[0].scatter(exam_score, grades_y)
axarr[0].set_title('Prijamni')
axarr[1].scatter(grade_in_highschool, grades_y)
axarr[1].set_title('Prosjek ocjena')
plt.show()

### b)

Naučite model L2-regularizirane regresije ($\lambda = 0.01$), na podacima `grades_X` i `grades_y`:

In [ ]:
model = Ridge(alpha=0.01)
model.fit(grades_X, grades_y)
print(model.coef_)

Sada ponovite gornji eksperiment, ali prvo skalirajte podatke `grades_X` i `grades_y` i spremite ih u varijable `grades_X_fixed` i `grades_y_fixed`. Za tu svrhu, koristite [`StandardScaler`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html).

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

grades_X_fixed = scaler.fit_transform(grades_X)
grades_y_fixed = scaler.fit_transform(grades_y.reshape(-1, 1))

model = Ridge(alpha=0.01)
model.fit(grades_X_fixed, grades_y_fixed)
print(model.coef_)

"""
f, axarr = plt.subplots(1, 2)
axarr[0].scatter(grades_X_fixed.T[0], grades_y_fixed)
axarr[0].set_title('Prijamni')
axarr[1].scatter(grades_X_fixed.T[1], grades_y_fixed)
axarr[1].set_title('Prosjek ocjena')
plt.show()
"""

**Q:** Gledajući grafikone iz podzadatka (a), koja značajka bi trebala imati veću magnitudu, odnosno važnost pri predikciji prosjeka na studiju? Odgovaraju li težine Vašoj intuiciji? Objasnite.  

### *7. Multikolinearnost

### a)

Izradite skup podataka `grades_X_fixed_colinear` tako što ćete u skupu `grades_X_fixed` iz
zadatka *7b* duplicirati zadnji stupac (ocjenu iz srednje škole). Time smo efektivno uveli savršenu multikolinearnost.

In [ ]:
grades_X_fixed_colinear = np.append(grades_X_fixed, grades_X_fixed.T[1].reshape(-1, 1), axis=1)

Ponovno, naučite na ovom skupu L2-regularizirani model regresije ($\lambda = 0.01$).

In [ ]:
model = Ridge(alpha=0.01)
model.fit(grades_X_fixed_colinear, grades_y)
print(model.coef_)

**Q:** Usporedite iznose težina s onima koje ste dobili u zadatku *7b*. Što se dogodilo?

### b)

Slučajno uzorkujte 50% elemenata iz skupa `grades_X_fixed_colinear` i naučite dva modela L2-regularizirane regresije, jedan s $\lambda=0.01$ i jedan s $\lambda=1000$). Ponovite ovaj pokus 10 puta (svaki put s drugim podskupom od 50% elemenata).  Za svaki model, ispišite dobiveni vektor težina u svih 10 ponavljanja te ispišite standardnu devijaciju vrijednosti svake od težina (ukupno šest standardnih devijacija, svaka dobivena nad 10 vrijednosti).

In [ ]:
model1 = Ridge(alpha=0.01)
model2 = Ridge(alpha=1000)
koef1 = []
koef2 = []

for i in range(10):
    indices = np.random.choice(grades_X_fixed_colinear.shape[0], 250, replace=False)
    model1.fit(grades_X_fixed_colinear[indices], grades_y[indices])
    model2.fit(grades_X_fixed_colinear[indices], grades_y[indices])

    print(f'Model 1: {model1.coef_}')
    print(f'Model 2: {model2.coef_}')

    koef1.append(model1.coef_)
    koef2.append(model2.coef_)

koef1 = np.array(koef1)
koef2 = np.array(koef2)

for i in koef1.T:
    print(f'Model 1 std: {np.std(i)}')

for i in koef2.T:
    print(f'Model 2 std: {np.std(i)}')

**Q:** Kako regularizacija utječe na stabilnost težina?  
**Q:** Jesu li koeficijenti jednakih magnituda kao u prethodnom pokusu? Objasnite zašto.